# Metage2Metabo

Metage2Metabo (m2m) is a python package to perform graph-based metabolic analysis starting from annotated genomes or metabolic networks.

This jupyter notebook uses the following versions of these dependencies:

- Metage2Metabo==1.6.0
- mpwt==0.6.0
- padmet==5.0.1
- MeneTools==3.4.0
- Miscoto==3.2.0
- bubbletools==0.6.11
- networkx==2.5
- pandas
- tabulate==0.9.0

In [1]:
from metage2metabo import __version__ as m2m_version
from menetools import __version__ as menetools_version
from miscoto import __version__ as miscoto_version
from networkx import __version__ as networkx_version
from pandas import __version__ as pandas_version
from tabulate import __version__ as tabulate_version

from importlib.metadata import version
padmet_version = version('padmet')
bubbletools_version = version('bubbletools')

print('Metage2Metabo: ' + m2m_version)
print('padmet: ' + padmet_version)
print('MeneTools: ' + menetools_version)
print('Miscoto: ' + miscoto_version)
print('networkx: ' + bubbletools_version)
print('networkx: ' + networkx_version)
print('pandas: ' + pandas_version)
print('tabulate: ' + tabulate_version)

Metage2Metabo: 1.5.4
padmet: 5.0.1
MeneTools: 3.3.0
Miscoto: 3.1.2
networkx: 0.6.11
networkx: 3.2.1
pandas: 2.1.4
tabulate: 0.9.0


## `m2m iscope`: Individual scope

This part of the workflow encompasses the computation of the individual scope.

### Scope formalisation

We define a metabolic network as a bipartite graph $G = (R \cup M, E)$, where $R$ and $M$ stand for reaction and metabolite nodes.  When $(m,r) \in E$ (respectively $(r,m) \in E$), with $m \in M$  and $r \in R$, the metabolite is called a _reactant_ (respectively _product_) of the reaction $r$. The scope of a set of seed compounds $S$ according to a metabolic network $G$, denoted by $Scope(G, S)$, is iteratively computed until it reaches a fixed point (Handorf and Ebenhöh 2005). It is formally defined by:

$$\Scope(G, S) = \bigcup_iM_i, \mbox{ where } M_0 = S \mbox{ and }
M_{i+1} = M_i \cup \Products{\{ r \in R \mid \Reactants\{r\} \subseteq M_i\}}.$$

### An example

In the following gif, we look at a metabolic network with 3 reactions (R1, R2, R3) and 6 metabolites (A, B, C, D, E, F).

The seeds are composed of metabolites A and B.

With these inputs, the scope will be computed. As A and B are available as seed, they can be used as reactant to activate the reaction R1, which will produce metabolite D. But the reaction R2 will not be activated as the metabolite C is not availabe in the seeds.

The metabolite D, produced by the reaction R1, can activate the reaction R3 to produce the metabolite F.

<p>
    <img src='images/iscope.gif' width='30%'>
    <center><em>Figure 1: Individual scope</em></center>
</p>

### Tutorial

For this tutorial, we will use an example of a community consisting of 6 bacteria, i.e. 6 metabolic networks. The medium used for the simulation consists of 2 metabolites, $S_1$ and $S_2$. Additionally, we are interested in the production of 4 metabolites, $H$, $C$, $E$ and $foo$. In the SBML files of the dataset, each metabolite has the prefix $M\_$, and a suffix corresponding to the compartment it belongs to, e.g. "M_S1_c" is the metabolite $S_1$ in the cytosol. 

The metabolic networks of `bact1`, `bact2`, `bact3`, `bact4`, `bact5` and `bact6` are depicted below:

<p>
    <img src='images/tiny_toy_networks.png' width='70%'>
    <center><em>Figure 2: Metabolic networks of the toy community</em></center>
</p>

In each of the networks, the compounds of the nutritional environmenent (i.e. the seeds) are depicted in yellow, and the metabolites we are interested in (i.e. the targets) are depicted in green.

Let's look at the individual scope of each metabolic network in the medium consisting of the two seeds. 

In [2]:
# Instanciate the logger
import logging

logging.basicConfig()

# Run the individual scope with metage2metabo on the folder of SBML using the seeds files.
# This is similar to the command:
# m2m iscope -n data/community -s data/seeds.sbml -o output_folder
from metage2metabo.m2m import individual_scope

union_of_iscopes = individual_scope.iscope('../../test/metabolic_data/tiny_toy/networks/', '../../test/metabolic_data/tiny_toy/seeds_community.sbml', 'output_folder')

## The scopes for each species will be computed in the output_folder. The union of all individual scopes will be returned as a set. 

print(union_of_iscopes)


{'M_N_c', 'M_D_c', 'M_H_c', 'M_A_c', 'M_R_c', 'M_S2_c', 'M_B_c'}


The union of the individual scope encompasses 7 metabolites. More details are provided in the output files. Let's look at them.

In [3]:
# Read the result

import json

with open('output_folder/indiv_scopes/indiv_scopes.json') as iscope_output:
    iscope_json_data = json.load(iscope_output)

print('Individual scope:')

print('{0}: {1} metabolites ({2}).'.format('bact1', len(iscope_json_data['bact1']), ', '.join(sorted(iscope_json_data['bact1']))))

Individual scope:
bact1: 5 metabolites (M_A_c, M_B_c, M_D_c, M_H_c, M_R_c).


We can check the individual scope of each network (circled in orange below):


<p>
    <img src='images/tiny_toy_iscopes.png' width='70%'>
    <center><em>Figure 3: Individual scopes</em></center>
</p>

In [4]:
for org in iscope_json_data.keys():
    print('{0}: {1} metabolites ({2}).'.format(org, len(iscope_json_data[org]), ', '.join(sorted(iscope_json_data[org]))))

bact5: 0 metabolites ().
bact3: 2 metabolites (M_A_c, M_B_c).
bact6: 0 metabolites ().
bact2: 4 metabolites (M_A_c, M_B_c, M_N_c, M_S2_c).
bact4: 1 metabolites (M_B_c).
bact1: 5 metabolites (M_A_c, M_B_c, M_D_c, M_H_c, M_R_c).


We notice that two species cannot produce anything from the seeds: `bact5` and `bact6`. 

Species `bact1` has the largest scope, 5 metabolites. 

Species `bact2` has the potential to produce one of the seeds, $S_2$. 

We notice that contrary to the initial definition of the scope above, only the seeds that can really be produced, i.e are the product of a reaction that is predicted to be activated, are included in the scopes. 

More generally, one might be interested in checking the status of the seeds, and take a look at the dedicated output:

In [5]:
with open('output_folder/indiv_scopes/seeds_in_indiv_scopes.json') as f:
    iscope_seeds_data = json.load(f)

print('Seeds that cannot be individually produced:')
print(iscope_seeds_data['individually_non_producible_seeds'])

print('\nSeeds that can be individually produced:')
print(iscope_seeds_data['individually_producible_seeds'])

print('\nSeeds that do not appear in the metabolic networks:')
print(iscope_seeds_data['seeds_absent_in_metabolic_network'])

Seeds that cannot be individually produced:
{'bact1': ['M_S2_c', 'M_S1_c'], 'bact2': ['M_S1_c'], 'bact3': ['M_S2_c', 'M_S1_c'], 'bact4': ['M_S2_c'], 'bact5': ['M_S1_c'], 'bact6': ['M_S1_c']}

Seeds that can be individually produced:
{'bact1': [], 'bact2': ['M_S2_c'], 'bact3': [], 'bact4': [], 'bact5': [], 'bact6': []}

Seeds that do not appear in the metabolic networks:
{'bact1': [], 'bact2': [], 'bact3': [], 'bact4': ['M_S1_c'], 'bact5': ['M_S2_c'], 'bact6': ['M_S2_c']}


Notable information here is that $S_1$ does not appear in `bact2` and $S_2$ does not appear in `bact5` nor `bact6`. A context in which a seed never occurs in any of the networks might indicate an error in the seed definition, a typo for instance. 

The fact that a species has the ability to produce a seed could be interesting to suggest a renewal of the seed, and less competition for that molecule. 

Other outputs related to the individual scope computation are the reversed iscopes. They provide the same information but with a focus on the molecules rather than the species: which species are predicted to produce each molecule? The information is stored as a json file and as a matrix.

In [6]:
with open('output_folder/indiv_scopes/rev_iscope.json') as f:
    rev_iscope = json.load(f)

print(json.dumps(rev_iscope, indent = 4, sort_keys=True))


{
    "M_A_c": [
        "bact3",
        "bact2",
        "bact1"
    ],
    "M_B_c": [
        "bact3",
        "bact2",
        "bact4",
        "bact1"
    ],
    "M_D_c": [
        "bact1"
    ],
    "M_H_c": [
        "bact1"
    ],
    "M_N_c": [
        "bact2"
    ],
    "M_R_c": [
        "bact1"
    ],
    "M_S2_c": [
        "bact2"
    ]
}


In [7]:
import pandas 
import csv
from io import StringIO

display(pandas.read_csv('output_folder/indiv_scopes/rev_iscope.tsv', sep='\t', index_col='Unnamed: 0'))

,M_A_c,M_B_c,M_N_c,M_S2_c,M_D_c,M_H_c,M_R_c
bact5,0,0,0,0,0,0,0
bact3,1,1,0,0,0,0,0
bact6,0,0,0,0,0,0,0
bact2,1,1,1,1,0,0,0
bact4,0,1,0,0,0,0,0
bact1,1,1,0,0,1,1,1


### Command line from the terminal

Most of the time, you will launch m2m from the terminal rather than from the API. 
The command for individual scope computation is the following, it will create all output files:

```sh
m2m iscope -n ../../test/metabolic_data/tiny_toy/networks/ -s ../../test/metabolic_data/tiny_toy/seeds_community.sbml -o output_folder
```


## `m2m cscope`: Community scope

This part encompasses the computation of the community scope, i.e. the metabolic potential of the community considering that interactions could occur between members. By interactions, we hereby mean cross feedings, or metabolic exchanges. We consider that any molecule can become a by-product or be shared through cross-feeding to other bacterial populations. We therefore assess the upper bound of the metabolic potential, considering that it is unlikely that all metabolites can be shared to all species. 

### Community scope formalisation

The `m2m cscope` command computes the metabolic capabilities of the whole microbiota by taking into account the complementarity of metabolic pathways between metabolic networks. This step simulates the sharing of metabolic biosynthesis through a meta-organism composed of all metabolic networks, and assesses the metabolic compounds that can be reached using network expansion. This calculation is an extension of the features of [MiSCoTo](https://github.com/cfrioux/miscoto) in which the collective scope of a collection of metabolic networks $\{G_1, \dots G_N\}$ is introduced. We define:

$$CommunityScope(G_1.. G_N, S)=Scope \left( \left(\bigcup_{i \in {\{1..n\}}} R_i, \bigcup_{i \in {\{1..n\}}} M_i, \bigcup_{i \in {\{1..n\}}} E_i \right), S \right).$$ 

### Application to the example dataset. 

In this example, we have 6 metabolic networks (`bact1`, `bact2`, `bact3`, `bact4`, `bact5` and `bact6`). As for the Individual Scope, the seeds are composed of two metabolites (S1 and S2) and highlighted in yellow. With these inputs, the Community Scope can be computed.

The formalism behind the community scope of m2m uses a Mixed Bag modelling, which considers the metabolic networks of the community as a meta-organism allowing exchanges between them without cost. For example, no reaction in the metabolic network `bact2` produces metabolite `C` as reactant but reactions of metabolic networks `bact5` and `bact6` does. With the Mixed Bag modelling, the metabolite `C` is then available to be used by `bact2` after being produced by `bact5` and `bact6`. 


The Community Scope of the 6 organisms is composed of of 15 metabolites, all circled in blue.

<p>
    <img src='images/tiny_toy_cscope.png' width='60%'>
    <center><em>Figure 4: Community scope</em></center>
</p>

We could visualise the meta-metabolism of an organism with all the merged reactions of the 6 species. 

<p>
    <img src='images/tiny_toy_cscope_metaorg.png' width='50%'>
    <center><em>Figure 5: Corresponding meta-organism sharing all metabolic capabilities</em></center>
</p>

In [8]:
# Run the community scope with metage2metabo on the folder of SBML using the seeds files.
# This is similar to the command:
#m2m cscope -n data/community -s data/seeds.sbml -o output_folder
from metage2metabo.m2m import community_scope

instance_path, network_scopes = community_scope.cscope('../../test/metabolic_data/tiny_toy/networks/', '../../test/metabolic_data/tiny_toy/seeds_community.sbml', 'output_folder')

print('The community scope contains {0} metabolites: {1}.'.format(len(network_scopes), ', '.join(sorted(network_scopes))))

The community scope contains 15 metabolites: M_A_c, M_B_c, M_C_c, M_D_c, M_E_c, M_F_c, M_G_c, M_H_c, M_K_c, M_N_c, M_R_c, M_S1_c, M_S2_c, M_V_c, M_X_c.


It is however important to know which species are responsible for the prodicibility of each molecule, i.e. has an activated reaction that produce the molecules (although the activation of this reaction might rely on interactions with other species). 

This information is also provided by `m2m cscope`, in a dedicated output file. 

Overall the community scope command creates 4 output files : 

- the list of the compounds in the community scope, stored in `output_folder/community_analysis/comm_scopes.json`
- the contribution of each microbe to the community scope in `output_folder/community_analysis/contributions_of_microbes.json`
- the reverse community scope, with a focus on the molecules rather that the species, in json format
- the reverse community scope, with a focus on the molecules rather that the species, in tabulated format

Let's look at the outputs:

First the community scope itself:


In [9]:
with open('output_folder/community_analysis/comm_scopes.json') as f:
    cscope = json.load(f)

print(json.dumps(cscope, indent = 4, sort_keys=True))

{
    "com_scope": [
        "M_N_c",
        "M_S1_c",
        "M_D_c",
        "M_V_c",
        "M_C_c",
        "M_H_c",
        "M_A_c",
        "M_K_c",
        "M_G_c",
        "M_E_c",
        "M_X_c",
        "M_F_c",
        "M_R_c",
        "M_S2_c",
        "M_B_c"
    ]
}


The contribution of microbes file describes what each community member produces individually (iscope), and in community, and highlights the added-value of interactions, i.e. what can be produced in community that cannot be produced individually.

In [10]:
with open('output_folder/community_analysis/contributions_of_microbes.json') as f:
    contribs = json.load(f)

print(json.dumps(contribs, indent = 4, sort_keys=True))

{
    "bact1": {
        "community_metabolic_gain": [
            "M_F_c"
        ],
        "produced_alone": [
            "M_R_c",
            "M_H_c",
            "M_A_c",
            "M_D_c",
            "M_B_c"
        ],
        "produced_in_community": [
            "M_H_c",
            "M_A_c",
            "M_B_c",
            "M_R_c",
            "M_D_c",
            "M_F_c"
        ]
    },
    "bact2": {
        "community_metabolic_gain": [
            "M_H_c",
            "M_G_c",
            "M_E_c"
        ],
        "produced_alone": [
            "M_B_c",
            "M_S2_c",
            "M_N_c",
            "M_A_c"
        ],
        "produced_in_community": [
            "M_G_c",
            "M_H_c",
            "M_A_c",
            "M_E_c",
            "M_B_c",
            "M_S2_c",
            "M_N_c"
        ]
    },
    "bact3": {
        "community_metabolic_gain": [
            "M_N_c",
            "M_S1_c",
            "M_H_c",
            "M_E_c",
        

The reverse community scopes is a dictionnary with compounds as keys and species that produce them in community as values.

In [11]:
with open('output_folder/community_analysis/rev_cscope.json') as f:
    rev_cscope = json.load(f)

print(json.dumps(rev_cscope, indent = 4, sort_keys=True))

{
    "M_A_c": [
        "bact3",
        "bact2",
        "bact1"
    ],
    "M_B_c": [
        "bact3",
        "bact2",
        "bact4",
        "bact1"
    ],
    "M_C_c": [
        "bact5",
        "bact6"
    ],
    "M_D_c": [
        "bact1"
    ],
    "M_E_c": [
        "bact3",
        "bact2"
    ],
    "M_F_c": [
        "bact1"
    ],
    "M_G_c": [
        "bact3",
        "bact2"
    ],
    "M_H_c": [
        "bact3",
        "bact2",
        "bact1"
    ],
    "M_K_c": [
        "bact5",
        "bact6"
    ],
    "M_N_c": [
        "bact3",
        "bact2"
    ],
    "M_R_c": [
        "bact1"
    ],
    "M_S1_c": [
        "bact3"
    ],
    "M_S2_c": [
        "bact2"
    ],
    "M_V_c": [
        "bact5"
    ],
    "M_X_c": [
        "bact6",
        "bact4"
    ]
}


The matrix of the community scope

In [12]:
display(pandas.read_csv('output_folder/community_analysis/rev_cscope.tsv', sep='\t', index_col='Unnamed: 0'))

,M_K_c,M_V_c,M_C_c,M_S1_c,M_B_c,M_N_c,M_G_c,M_H_c,M_A_c,M_E_c,M_X_c,M_S2_c,M_R_c,M_D_c,M_F_c
bact5,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
bact3,0,0,0,1,1,1,1,1,1,1,0,0,0,0,0
bact6,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0
bact2,0,0,0,0,1,1,1,1,1,1,0,1,0,0,0
bact4,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
bact1,0,0,0,0,1,0,0,1,1,0,0,0,1,1,1


### Command line from the terminal

Most of the time, you will launch m2m from the terminal rather than from the API. 
The command for community scope computation is the following, it will create all output files:

```sh
m2m cscope -n ../../test/metabolic_data/tiny_toy/networks/ -s ../../test/metabolic_data/tiny_toy/seeds_community.sbml -o output_folder
```


## `m2m addedvalue`: Cooperation potential

This part encompasses the computation of the cooperation potential, i.e. the set of molecules that can be produced in community but not individually given the seeds provided as input.

### Cooperation potential formalisation

Given individual and community metabolic potentials, the _cooperation potential_ consists in the set of metabolites whose producibility can only occur if several organisms participate in the biosynthesis. `m2m addedvalue` computes the cooperation potential by performing a set difference between the community scope and the union of individual scopes, and produces a SBML file with the resulting metabolites. This list of compounds is inclusive and could comprise false positives not necessitating cooperation for production, but selected due to missing annotations in the initial genomes. One can modify the SBML file accordingly, prior to the following M2M community reduction step.

The cooperation potential $\coopPot(G_1,..,G_n,S)$ of a collection of metabolic networks $\{G_1..G_n\}$ is defined by:


$\coopPot(G_1,..,G_n,S) = \mixedbagScope(G_1,..,G_n,S)  \setminus  \bigcup_{i \in {\{1..n\}}} \Scope(G_i,S).$

### An example

In the **Figure 4**, the cooperation potential is:

$\coopPot(bact1, bact2, bact3, bact4, bact5, bact6) = \{A, B, C, D, E, F, G, H, K, N, R, S2, S1, V, X\} - \{A, B, D, H, N, R, S2\} $

$\coopPot(OrgA,OrgB,OrgC,S) = \{C, E, F, G, H, K, S1, V, X\} $



In [13]:
# Run the cooperation potential with metage2metabo on the folder of SBML using the seeds files.
# This is similar to the command:
# m2m addedvalue -n data/community -s data/seeds.sbml -o output_folder
from metage2metabo.m2m import community_addedvalue, individual_scope, community_scope

networks_path = '../../test/metabolic_data/tiny_toy/networks/'
seeds_path = '../../test/metabolic_data/tiny_toy/seeds_community.sbml'
output_folder = 'output_folder'

iscope_metabolites = individual_scope.iscope(networks_path, seeds_path, output_folder)
instance_comscope, cscope_metabolites = community_scope.cscope(networks_path, seeds_path, output_folder)

addedvalue = community_addedvalue.addedvalue(iscope_metabolites, cscope_metabolites, output_folder)

In [14]:
print('The union of individual scopes contains {0} metabolites: {1}.'.format(len(iscope_metabolites), ', '.join(sorted(iscope_metabolites))))
print('The community scope contains {0} metabolites: {1}.'.format(len(cscope_metabolites), ', '.join(sorted(cscope_metabolites))))
print('The cooperation potential contains {0} metabolites: {1}.'.format(len(addedvalue), ', '.join(sorted(addedvalue))))

The union of individual scopes contains 7 metabolites: M_A_c, M_B_c, M_D_c, M_H_c, M_N_c, M_R_c, M_S2_c.
The community scope contains 15 metabolites: M_A_c, M_B_c, M_C_c, M_D_c, M_E_c, M_F_c, M_G_c, M_H_c, M_K_c, M_N_c, M_R_c, M_S1_c, M_S2_c, M_V_c, M_X_c.
The cooperation potential contains 8 metabolites: M_C_c, M_E_c, M_F_c, M_G_c, M_K_c, M_S1_c, M_V_c, M_X_c.


### Command line from the terminal

Most of the time, you will launch m2m from the terminal rather than from the API. 
The command for added value computation is the following, it will create all output files:

```sh
m2m addedvalue -n ../../test/metabolic_data/tiny_toy/networks/ -s ../../test/metabolic_data/tiny_toy/seeds_community.sbml -o output_folder
```


## `m2m mincom`: Minimal community selection


### Minimal community formalism

A minimal community $\mathcal{C}$ enabling the producibility of a set of targets $T$ from the seeds $S$ is a sub-family of the community $G_1, \dots, G_n$ which is solution of the following optimisation problem:

$SetofMinimalCommunity(G_1 .. G_n, S, T) =  arg \, min ( size( {\{G_i..G_j\}} \, | \, \mathcal{P}_{\{G_1..G_n\}} \subset  \Sigma_{\{G_i..G_j\}}(S) )$

$$\begin{array}{lll}
 \qquad \qquad  & 
 \displaystyle{
 \minimize_{ \{G_{i_1}.. G_{i_L}\}  \  \subset \{G_{1}.. G_{N}\} }
 }
  & \mbox{size} (\{G_{i_1}.. G_{i_L}\})   \\ 
& \mbox{subject to }  &   T \subset  \mixedbagScope(G_{i_1} .. G_{i_L}),S). 
\end{array}       
$$

### Example

In this example, we have 8 species. In order to produce the targets from the seeds, there are 4 minimal communities of size of 3 organisms.

The yellow organism occurs in all minimal communities, it is an essential symbiont.
Green, orange, cyan and purple organisms occur in some minimal communities but not in all, they are alternatives symbionts.

<p>
    <img src='images/key_species.png' width='40%'>
    <center><em>Figure 6: Key species</em></center>
</p>


### Key species

Many minimal communities are expected to be equivalent for a given metabolic objective but their enumeration can be computationally costly.

We define *key species* which are organisms occurring in at least one community among all the optimal ones. Key species can be further distinguished into *essential symbionts* and *alternative symbionts*. The former occur in every minimal community whereas the latter occur only in some minimal communities. More precisely, the key species $\ks(G_1 .. G_n, S,T)$, the essential symbionts $\es(G_1 .. G_n, S,T)$, and the alternative symbionts $\as(G_1 .. G_n, S,T)$ associated to a set of metabolic networks, seeds $S$ and a set of target metabolites $T$ are defined by:


\begin{align*}
    \ks(G_1 .. G_n, S,T) &= \{ G \mid \exists {\mathcal C} \in \mincom(G_1 .. G_n, S, T), \, G \in {\mathcal C}\}. \\
    \es(G_1 .. G_n, S,T) &= \{ G \mid \forall {\mathcal C} \in \mincom(G_1 .. G_n, S, T), \, G \in {\mathcal C}\}. \\
    \as(G_1 .. G_n, S,T) &= \ks(G_1 .. G_n, S,T) \setminus \es(G_1 .. G_n, S,T).
\end{align*}

In [15]:
# Run the minimal community with metage2metabo on the folder of SBML using the seeds files.
# This is similar to the command:
#m2m metacom -n data/community -s data/seeds.sbml -o output_folder
from metage2metabo.m2m import community_addedvalue, individual_scope, community_scope, minimal_community
from metage2metabo import sbml_management

networks_path = '../../test/metabolic_data/tiny_toy/networks/'
seeds_path = '../../test/metabolic_data/tiny_toy/seeds_community.sbml'
output_folder = 'output_folder'
target_path = '../../test/metabolic_data/tiny_toy/targets_community.sbml'

iscope_metabolites = individual_scope.iscope(networks_path, seeds_path, output_folder)
instance_comscope, cscope_metabolites = community_scope.cscope(networks_path, seeds_path, output_folder)
addedvalue = community_addedvalue.addedvalue(iscope_metabolites, cscope_metabolites, output_folder)

instance_mincom = community_scope.instance_community(networks_path, seeds_path, output_folder, target_path)

# The path contains the instance for Answer Set Programming computation of minimal communities.
print(instance_mincom)

# let's run mincom

minimal_community.mincom(instance_mincom, seeds_path, target_path, output_folder)

/home/abelcour/Documents/Softwares/git/metage2metabo/tutorials/method_tutorial/output_folder/community_analysis/miscoto_6dr7c6nv.lp
/home/abelcour/.local/lib/python3.10/site-packages/miscoto/encodings/community_soup.lp


The mincom command creates the output `community_analysis/mincom.json`. 

When called with the command line, logs provide an explanation of the results:

```
m2m mincom -n ../../test/metabolic_data/tiny_toy/networks -s ../../test/metabolic_data/tiny_toy/seeds_community.sbml -t ../../test/metabolic_data/tiny_toy/targets_community.sbml -o output_folder
######### Creating metabolic instance for the whole community #########
Created temporary instance file in output_folder/community_analysis/miscoto__3lz5o7z.lp

###############################################
#                                             #
#         Minimal community selection         #
#                                             #
###############################################

Running minimal community selection
/Users/cfrioux/.pyenv/versions/metage2metabo/lib/python3.10/site-packages/miscoto/encodings/community_soup.lp

In the initial and minimal communities 3 targets are producible and 1 remain unproducible.

3 producible targets:
M_F_c
M_C_c
M_H_c

1 still unproducible targets:
M_foo_c

Minimal communities are available in output_folder/community_analysis/mincom.json

######### One minimal community #########
# One minimal community enabling the producibility of the target metabolites given as inputs
Minimal number of bacteria in communities => 3

bact5
bact1
bact3
######### Key species: Union of minimal communities #########
# Bacteria occurring in at least one minimal community enabling the producibility of the target metabolites given as inputs
Number of key species => 5

bact1
bact5
bact2
bact6
bact3
######### Essential symbionts: Intersection of minimal communities #########
# Bacteria occurring in ALL minimal communities enabling the producibility of the target metabolites given as inputs
Number of essential symbionts => 1

bact1
######### Alternative symbionts: Difference between Union and Intersection #########
# Bacteria occurring in at least one minimal community but not all minimal communities enabling the producibility of the target metabolites given as inputs
Number of alternative symbionts => 4

bact2
bact3
bact6
bact5

--- Mincom runtime 4.99 seconds ---

--- Total runtime 5.06 seconds ---

```

The figure below illustrates the one solution that mincom provides, with the underlying minimal metabolic exchanges that would be necessary (can be computed with the `miscoto` package, a dependency of m2m).

<p>
    <img src='images/tiny_toy_mincom_onesol.png' width='60%'>
    <center><em>Figure 6: One minimal solution enabling the production of the 3 producible targets</em></center>
</p>

One target cannot be produced. It is the $foo$ metabolite that actually occurs in none of the metabolic networks. 

More information can be computed regarding target producibility.
It is done automatically when calling `m2m metacom`, e.g. `m2m metacom -n ../../test/metabolic_data/tiny_toy/networks -s ../../test/metabolic_data/tiny_toy/seeds_community.sbml -t ../../test/metabolic_data/tiny_toy/targets_community.sbml -o output_folder` and is stored in `output_folder/producibility_targets.json`.


The file `producibility_targets.json` provides such information, stored as a dictionnary:

- Producible targets
  ```
    "producible": [
        "M_H_c",
        "M_C_c",
        "M_F_c"
  ```
- Unproducible targets
  ```
      "unproducible": [
        "M_foo_c"
    ],
  ```
- Targets that can be produced by individual species, i.e. do not need cooperation events, together with their producers:
  ```
    "indiv_producible": [
        "M_H_c"
    ],
    "individual_producers": {
        "M_H_c": [
            "bact1"
        ]
    },
  ```
- Targets that requires cooperation or cross-feeding for their production, together with their final producers
   ```
    "com_only_producers": {
        "M_H_c": [
            "bact3",
            "bact2"
        ],
        "M_F_c": [
            "bact1"
        ],
        "M_C_c": [
            "bact6",
            "bact5"
        ]
    },
   ```
- Key species
  ```
    "key_species": [
        "bact3",
        "bact2",
        "bact1",
        "bact6",
        "bact5"
    ],
  ```
- Targets producible in the minimal community
  ```
     "mincom_producible": [
        "M_H_c",
        "M_C_c",
        "M_F_c"
    ],
  ```


Please note that for computational efficiency, only one solution, the key species and essential/alternative symbionts are computed. Enumerating all equivalent minimal communities takes more time but can be computed. That is the purpose of `m2m analysis`.

### Command line from the terminal

Most of the time, you will launch m2m from the terminal rather than from the API. 
The command for minimal community computation is the following, it will create all output files:

```sh
m2m mincom -n ../../test/metabolic_data/tiny_toy/networks/ -s ../../test/metabolic_data/tiny_toy/seeds_community.sbml -t ../../test/metabolic_data/tiny_toy/targets_community.sbml -o output_folder
```

`m2m metacom` runs the complete analysis from the individual scope to the computation of target producibility after minimal community selection. 
We advise to use directly this command to explore the metabolic potential of communities. 

```sh
m2m metacom -n ../../test/metabolic_data/tiny_toy/networks/ -s ../../test/metabolic_data/tiny_toy/seeds_community.sbml -t ../../test/metabolic_data/tiny_toy/targets_community.sbml -o output_folder
```

## `m2m_analysis`: Visualisation of minimal communities

To complete the analysis of minimal communities and visualise the results, we will use the m2m_analysis part of m2m. This part is more computational heavy due to the fact that it will enumerate all minimal communities (especially it can need more RAM).

Then using the results of enumeration, it will try to create visualisation of the minimal communities.

In [16]:
from metage2metabo.m2m_analysis import m2m_analysis_workflow

# we need a target file in which all molecules can be producible. 
# foo is not producible, so we'll use another target file that does not contain it

networks_path = '../../test/metabolic_data/tiny_toy/networks/'
seeds_path = '../../test/metabolic_data/tiny_toy/seeds_community.sbml'
output_folder = 'output_folder'
target_path = '../../test/metabolic_data/tiny_toy/targets_community_allprod.sbml'

m2m_analysis_workflow.run_analysis_workflow(networks_path, target_path, seeds_path, output_folder, None, None)

As seen in part `m2m mincom`, we have mminimal communities composed of 3 organisms. Now using `m2m_analysis`, we will enumerate these different minimal communties.

Minimal communities are described in `output_folder/json/targets_community_allprod.json`.
There are 4 minimal communities:

- **`bact1`** X `bact3` x `bact6`
- **`bact1`** X `bact3` x `bact5`
- **`bact1`** X `bact2` x `bact6`
- **`bact1`** X `bact2` x `bact5`

The 4 communities are illustrated below:

<p>
    <img src='images/tiny_toy_enum_sol.png' width='60%'>
    <center><em>Figure 7: All minimal community solutions</em></center>
</p>

Then, `m2m_analysis` will compute the solution graph. This graph's nodes correspond to organism from the minimal community. An edge is drawn between two nodes if the two organisms occur in the same minimal community.

This graph can be very dense and hard to visualise if a lot of minimal solutions exist. 
It is stored in `output_folder/gml/targets_community_allprod.gml

<p>
    <img src='images/tiny_toy_association_graph.png' width='40%'>
    <center><em>Figure 7: All minimal community solutions</em></center>
</p>

Then m2m_analysis will compress the solution graph into a [power graph](https://en.wikipedia.org/wiki/Power_graph_analysis). To do this, m2m_analysis uses the `powergrasp` package, which will search for specific pattern (like hubs or cliques) and compress them into power nodes and power edges. Here, as node `bact1` is a hub in the graph (as connected to the other nodes of the graph). We notice that `bact2` and `bact3` have the same roles in minimal communities, likewise for `bact5` and `bact6`. Each pair will belong to a power node, indicating they can replace each other, i.e. a minimal community holds one of the two pair elements. 

`bact1` occurs in all the minimal communities, it is an essential symbiont (in dark pink).

`bact2`, `bact3`, `bact5` and `bact6` occur in at least one minimal community but not all, they are alternative symbionts (in blue).

<p>
    <img src='images/tiny_toy_powergraph.png' width='40%'>
    <center><em>Figue 9: Powergraph</em></center>
</p>

The power graph information is stored in `output_folder/bbl`. It can be visualised as an html page `html/targets_community_allprod_powergraph.html`.

### Command line from the terminal

Most of the time, you will launch m2m_analysis from the terminal rather than from the API. 
The command is the following, it will create all output files:

```sh
m2m_analysis workflow -n ../../test/metabolic_data/tiny_toy/networks/ -s ../../test/metabolic_data/tiny_toy/seeds_community.sbml -t ../../test/metabolic_data/tiny_toy/targets_community_allprod.sbml -o output_folder
```

Other options are available, for instance for creating the svg image of the power graph, or taking into account the taxonomy of the community members. Take a look at the command line help `m2m_analysis workflow --help` and the documentation.

Here the whole workflow is run. You can launch parts of the workflow, take a look at the documentation and the help of the tool using `m2m_analysis -h`.

The command produces the following logs:

```
###############################################
#                                             #
#      Enumeration of minimal communities     #
#                                             #
###############################################

######### Enumeration of solution for: targets_community_allprod #########
/shared/Softwares/git/miscoto/miscoto/encodings/community_soup.lp
######### Enumeration of minimal communities #########
4 minimal communities (each containing 3 species) producing the target metabolites
######### Key species: Union of minimal communities #########
# Bacteria occurring in at least one minimal community enabling the producibility of the target metabolites given as inputs
Key species = 5
bact1
bact5
bact2
bact3
bact6
######### Essential symbionts: Intersection of minimal communities #########
# Bacteria occurring in ALL minimal community enabling the producibility of the target metabolites given as inputs
Essential symbionts = 1
bact1
######### Alternative symbionts: Difference between Union and Intersection #########
# Bacteria occurring in at least one minimal community but not all minimal community enabling the producibility of the target metabolites given as inputs
Alternative symbionts = 4
bact3
bact6
bact2
bact5
--- Enumeration runtime 0.79 seconds ---


###############################################
#                                             #
#         Solution graph creation             #
#                                             #
###############################################

######### Graph of targets_community_allprod #########
Number of nodes: 5
Number of edges: 8
--- Graph runtime 0.01 seconds ---


###############################################
#                                             #
#  Compression and visualisation of graph     #
#                                             #
###############################################

######### Graph compression: targets_community_allprod #########
Number of powernodes: 3
Number of poweredges: 2
Compression runtime 1.43 seconds ---

######### Test powergraph heuristics: targets_community_allprod #########
Same combinations between theorical (4) and solution (4)
The powergraph seems to be an optimal representation of the solutions.

Same combinations between theorical (4) and the enumeration of solutions by m2m_analysis  (4)
The powergraph seems to be an optimal representation of the solutions.

Same number of solution between the computed combinations from powernodes (4) and the enumeration of solutions by m2m_analysis (4)
But this does not indicate that the powernodes are an optimal representation but that they contain the solution.

It seems that there are no heuristics in powergraph so it could be possible to create a boolean equation.
######### Boolean equation of minimal communities #########
The boolean equation can only be created for simple case (without too many combinations).
Boolean equation seems good, as it has the same combinations (4) than the one from enumeration (4).
Boolean equation: 
(( bact1 ) & 
( bact6 | bact5 ) & 
( bact3 | bact2 ) )
######### PowerGraph visualization: targets_community_allprod #########
Creation of the powergraph website accessible at output_folder/html/targets_community_allprod
--- Powergraph runtime 1.44 seconds ---

--- m2m_analysis runtime 2.24 seconds ---

--- Total runtime 2.24 seconds ---

```

Graph compression can used an heuristics if the combination are too complex. If this is the cas, `m2m_analysis` will write warning in the log during the `Test powergraph heuristics:` step.

Furthermore, this will prevent the creation of a boolean equation summarizing the minimal communities.

But if the combination are simple a boolean equation can be created. The powergraph is then sumarized into a boolean equation:

`(( bact1 ) & ( bact3 | bact2 ) & ( bact6 | bact5 ) )`

`&` corresponds to `AND` logical operator. `|` corresponds to `OR` logical operator. If an organism is alone between parenthesis, it is an essential symbiont. If it is within parenthesis with other organisms and separated by `|`, it is an alternative symbiont.

It reads as follow: to produce the target metabolites we need `bact1` and either `bact3 or bact2` and either `bact6 or bact5`.

# Tutorial on real data

For this tutorial, we used a subset of the data used in the article of M2M (avaible in the [Github repository](https://github.com/AuReMe/metage2metabo/tree/master/test/metabolic_data)).

- `toy_bact`: 17 metabolic networks
- `seeds_toy.sbml`: 93 metabolites (classical diet for the gut microbiota, EU average from the VMH resource and a small number of currency metabolites).



In [17]:
# Instanciate the logger
import logging
import statistics
import json
import os

logging.basicConfig()
logging.setLevel = logging.CRITICAL
# Run the individual scope with metage2metabo on the folder of SBML using the seeds files.
# This is similar to the command:
# m2m iscope -n data/community -s data/seeds.sbml -o output_folder
from metage2metabo.m2m import individual_scope

networks_path = '../../test/metabolic_data/toy_bact'
seeds_path = '../../test/metabolic_data/seeds_toy.sbml'
output_folder = 'tutorial_output_folder'
indiv_output = os.path.join(output_folder, 'indiv_scopes')
indiv_scope_json = os.path.join(indiv_output, 'indiv_scopes.json')
key_species_file = os.path.join(output_folder, 'key_species.json')
target_path = '../../test/metabolic_data/targets_toy.sbml'

individual_scope.iscope(networks_path, seeds_path, output_folder)

# Read the result
with open(indiv_scope_json) as iscope_output:
    iscope_json_data = json.load(iscope_output)

metabolites = []
union_metabolites = set()
for species in iscope_json_data:
    metabolites.append(len(iscope_json_data[species]))
    union_metabolites.update(iscope_json_data[species])

print('Average individual scope: {0}'.format(statistics.mean(metabolites)))

# Run the community scope with metage2metabo on the folder of SBML using the seeds files.
# This is similar to the command: m2m cscope -n toy_bact -s seeds_toy.sbml -o output_folder
from metage2metabo.m2m import community_scope

instance_path, network_scopes = community_scope.cscope(networks_path, seeds_path, output_folder)

print('The community scope contains {0} metabolites.'.format(len(network_scopes)))

print('Cooperation potential: {0} metabolites.'.format(len(set(network_scopes)-union_metabolites)))
# Run the minimal community with metage2metabo on the folder of SBML using the seeds files.
# This is similar to the command: m2m metacom -n toy_bact -s seeds_toy.sbml -o output_folder
from metage2metabo.m2m import community_addedvalue, individual_scope, community_scope
from metage2metabo import sbml_management

iscope_metabolites = individual_scope.iscope(networks_path, seeds_path, output_folder)
instance_comscope, cscope_metabolites = community_scope.cscope(networks_path, seeds_path, output_folder)
addedvalue = community_addedvalue.addedvalue(iscope_metabolites, cscope_metabolites, output_folder)

if len(addedvalue):
    sbml_management.create_species_sbml(addedvalue, target_path)
instance_mincom = community_scope.instance_community(networks_path, seeds_path, output_folder, target_path)

from metage2metabo.m2m_analysis import m2m_analysis_workflow

m2m_analysis_workflow.run_analysis_workflow(networks_path, target_path, seeds_path, output_folder, None, None)

with open(key_species_file) as json_key_species:
    key_species_json_data = json.load(json_key_species)

nb_essential_symbionts = len(key_species_json_data['targets_toy']['essential_symbionts']['data'])
essential_symbionts = ','.join(key_species_json_data['targets_toy']['essential_symbionts']['data'])
nb_alternative_symbionts = len(key_species_json_data['targets_toy']['alternative_symbionts']['data'])
alternative_symbiotns = ','.join(key_species_json_data['targets_toy']['alternative_symbionts']['data'])
print('\n')
print('{0} Essential symbionts: {1}'.format(nb_essential_symbionts, essential_symbionts))
print('{0} Alternative symbionts: {1}'.format(nb_alternative_symbionts, alternative_symbiotns))

boolean_equation_json_file = os.path.join(output_folder, 'minimal_equation', 'targets_toy', 'boolean_equation.json')
with open(boolean_equation_json_file) as open_boolean_equation_json:
    boolean_equation_data = json.load(open_boolean_equation_json)

print('\n')
print('Boolean equation: {0}'.format(boolean_equation_data['boolean_equation']))

Average individual scope: 239.05882352941177
The community scope contains 698 metabolites.
Cooperation potential: 122 metabolites.


12 Essential symbionts: GCA_003437715,GCA_003437195,GCA_003438055,GCA_003437815,GCA_003437885,GCA_003437295,GCA_003437055,GCA_003437255,GCA_003437375,GCA_003437665,GCA_003437905,GCA_003437595
5 Alternative symbionts: GCA_003437345,GCA_003437325,GCA_003437175,GCA_003437785,GCA_003437945


Boolean equation: (( GCA_003437715 ) & ( GCA_003437945 | GCA_003437325 | GCA_003437175 | GCA_003437345 | GCA_003437785 ) & ( GCA_003437195 ) & ( GCA_003438055 ) & ( GCA_003437815 ) & ( GCA_003437885 ) & ( GCA_003437295 ) & ( GCA_003437055 ) & ( GCA_003437255 ) & ( GCA_003437375 ) & ( GCA_003437665 ) & ( GCA_003437905 ) & ( GCA_003437595 ) )


If you want more information on the output files:

- result files of m2m are described [here](https://metage2metabo.readthedocs.io/en/latest/output.html).
- result files of m2m_analysis are explained [here](https://metage2metabo.readthedocs.io/en/latest/m2m_analysis.html#m2m-analysis-output-files).